In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('dati.csv')
df.drop(columns='Unnamed: 0',inplace = True)

C:\Users\mikdr\AppData\Local\Temp\ipykernel_39300\2641605486.py:1: DtypeWarning: Columns (20,21,25,26,27,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dati.csv')


In [5]:
# we get rid of some useless columns 
cols = ['Gruppo','NUM_RISERVATA', 'Confermato','Progressivo'] 
df = df.drop(cols, axis=1) 

In [6]:
# Mettiamo 1 dove una persona coinvolta è morta dopo l'incidente
def replace_with_1(value):
    if type(value) == float:
        if value == 1.0:
            return 1
        else:
            return 0
    if value.startswith('DECEDUTO'):
        return 1
    else:
        return 0
df['DecedutoDopo'] = df['DecedutoDopo'].apply(replace_with_1)
print(df['DecedutoDopo'].unique())

[0 1]


In [7]:
# Mettiamo 1 dove una persona coinvolta è morta nell'incidente
def replace_with_1_deceduto(value):
    if value == -1 or value=='-1':
        return 1
    else:
        return 0
df['Deceduto'] = df['Deceduto'].apply(replace_with_1_deceduto)
print(df['Deceduto'].unique())

[0 1]


In [8]:
#Creo una nuova colonna con 1 se la persone è morta, 0 se non lo è
df['Deceduti'] = df['Deceduto']+df['DecedutoDopo']
print(df['Deceduti'].unique())

[0 1]


In [9]:
df = df.drop(columns=['DecedutoDopo','Deceduto'])

In [10]:
df = df.drop(df[df['NUM_FERITI'] == 'Ore Diurne'].index)
df = df.drop(df[df['NUM_FERITI'] == 'Sufficiente'].index)

In [11]:
df['NUM_FERITI'] = df['NUM_FERITI'].astype(int)

In [12]:
def definisci_ferito(value):
    if type(value) == float:
        return 0
    if value.startswith('Illeso'):
        return 0
    if value in ['Rimandato', 'Ricoverato', 'Rifiuta cure immediate', 'Prognosi riservata', 'Deceduto durante prime cure', 'Deceduto durante trasporto', 'Deceduto sul posto']:
        return 1
    else:
        return 0
df['Ferito'] = df['Tipolesione'].apply(definisci_ferito)
for valore in df['Ferito'].unique():
    print(valore)

0
1


In [13]:
df = df.drop(columns=['Tipolesione'])

In [14]:
fields = ['Protocollo', 'DataOraIncidente', 'Localizzazione1', 'STRADA1', 'NaturaIncidente', 'particolaritastrade','Segnaletica','CondizioneAtmosferica','Traffico', 'Visibilita', 'Illuminazione', 'Latitudine', 'Longitudine', 'Ferito', 'Deceduti' ]
df = df[fields]

In [15]:
df.rename(columns = {'DataOraIncidente':'Data','Localizzazione1':'Localizzazione','STRADA1':'Strada', 'particolaritastrade':'ParticolaritaStrade'}, inplace=True)   

In [16]:
print(df.columns)

Index(['Protocollo', 'Data', 'Localizzazione', 'Strada', 'NaturaIncidente',
       'ParticolaritaStrade', 'Segnaletica', 'CondizioneAtmosferica',
       'Traffico', 'Visibilita', 'Illuminazione', 'Latitudine', 'Longitudine',
       'Ferito', 'Deceduti'],
      dtype='object')


In [17]:
def sostituisci_nan(value):
    if type(value) == str:
        if value == 'nan':
            return 'Assente'
        else:
            return value
    if type(value) == float:
        return 'Assente'
    if value.isna():
        return 'Assente'

df['Localizzazione'] = df['Localizzazione'].apply(sostituisci_nan)

In [18]:
df['NaturaIncidente'] = df['NaturaIncidente'].apply(sostituisci_nan)

In [19]:
df['ParticolaritaStrade'] = df['ParticolaritaStrade'].apply(sostituisci_nan)

In [20]:
df['Segnaletica'] = df['Segnaletica'].apply(sostituisci_nan)

In [21]:
df['CondizioneAtmosferica'] = df['CondizioneAtmosferica'].apply(sostituisci_nan)

In [22]:
df['Traffico'] = df['Traffico'].apply(sostituisci_nan)

In [23]:
df['Visibilita'] = df['Visibilita'].apply(sostituisci_nan)

In [24]:
df['Illuminazione'] = df['Illuminazione'].apply(sostituisci_nan)

In [25]:
print(df.shape)

(1259203, 15)


In [26]:
df = df.dropna(subset=['Latitudine'])
df = df.dropna(subset=['Longitudine'])

In [27]:
conteggio_nulli = df['Latitudine'].isna().sum()
print(conteggio_nulli)

0


In [28]:
conteggio_nulli = df['Longitudine'].isna().sum()
print(conteggio_nulli)

0


In [29]:
print(df.shape)

(557481, 15)


In [30]:
split_values = df['Data'].str.split(' ')
df['Data'] = split_values.str[0]


In [31]:
print(df.columns)

Index(['Protocollo', 'Data', 'Localizzazione', 'Strada', 'NaturaIncidente',
       'ParticolaritaStrade', 'Segnaletica', 'CondizioneAtmosferica',
       'Traffico', 'Visibilita', 'Illuminazione', 'Latitudine', 'Longitudine',
       'Ferito', 'Deceduti'],
      dtype='object')


In [34]:
#salvo file csv
df.to_csv('dati_per_persona.csv', index="True")